## Demand Forecasting: Regression Analysis and Model Training

### 1. Problem Statement
- The energy industry is undergoing a transformative journey, marked by rapid modernization and technological advancements. Infrastructure upgrades, integration of intermittent renewable energy sources, and evolving consumer demands are reshaping the sector. However, this progress comes with its challenges. Supply, demand, and prices are increasingly volatile, rendering the future less predictable. Moreover, the industry's traditional business models are being fundamentally challenged. In this competitive and dynamic landscape, accurate decision-making is pivotal. The industry relies heavily on probabilistic forecasts to navigate this uncertain future, making innovative and precise forecasting methods essential that aids stakeholders in making strategic decisions amidst the shifting energy landscape. 

In [ ]:
import warnings
warnings.filterwarnings("ignore")

### 2. Data Ingestion

#### [Zenml-MLfow pipeline for Data Ingestion](../steps/demand_forecasting/data_ingestion.py)

#### 2.1 Import Data and Required Packages
- Importing Pandas, Numpy, Matplotlib, Seaborn, Scikit-learn 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import joblib



#### 2.2 Import the CSV Data as Pandas DataFrame
- Importing both Demand and Weather Data of Demand Forecasting and merging them

In [ ]:
df_demand = pd.read_csv('../dataset/Demand Forecasting/Demand Forecasting Demand Data upto Feb 21.csv', sep=',')
df_weather = pd.read_csv('../dataset/Demand Forecasting/Demand Forecasting Weather Data upto Feb 28.csv', sep=',')
df_merged=pd.merge(left=df_demand,right=df_weather, on='datetime')


### 3. Data Preprocessing and Visualizations

#### [Zenml-MLfow pipeline for Data Preprocessing](../steps/demand_forecasting/data_preprocessing.py)

#### 3.1 Show Top 5 Records
 - Showing top 5 and last 5 records


In [ ]:
df_merged.head()

In [ ]:
df_merged.tail()

#### 3.2 Checking if Unamed columns have any data
- Checking the data in unnamed columns and removing all the empty columns

In [ ]:
for i in range(21, 26):
    column_name = f'Unnamed: {i}'
    count_non_null = df_merged[column_name].notna().sum()
    print(f"Non-null values in {column_name}: {count_non_null}")

In [ ]:
columns_to_drop = ['Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25']
df_merged.drop(columns_to_drop, inplace=True, axis=1)

#### 3.3 Performing Datachecks
- Checking for null values

In [ ]:
df_merged.info()

#### 3.4 Filling most appropriate values for severerisk 
- Filling severerisk with 0 on nan values for more appropraite correlation analysis

In [ ]:
df_merged['severerisk'].fillna(0, inplace=True)

#### 3.5 Dropping redundant data
- Dropping preciptype and precipprob as precipitation has more accurate and non-null data, similarly dropping windgust and keeping windspeed

In [ ]:
df_merged.drop(["precipprob", "preciptype" ], inplace=True, axis=1)
df_merged.drop(['windgust'], inplace=True, axis=1)

In [ ]:
df_merged

#### 3.6 Interpolation of data
- Using .interpolate() method to add most appropriate datas in place of NaN values

In [ ]:
for column in df_merged.columns[3:17]:
    df_merged[column] = df_merged[column].interpolate(method='linear', limit_direction='forward', axis=0)

#### 3.7 Histogram & KDE
 - It is evident that the distribution of the 'Demand (MW)' column in the dataset closely aligns with a log-normal distribution.

In [ ]:
df_merged['datetime'] = pd.to_datetime(df_merged['datetime'])

fig, ax = plt.subplots(figsize=(10, 6))

sns.histplot(df_merged['Demand (MW)'], kde=False, bins=30, color='skyblue', ax=ax)
ax.set_title(f'Histogram of Demand (MW)')
ax.set_xlabel('Demand (MW)')
ax.set_ylabel('Frequency')

ax2 = ax.twinx()
sns.kdeplot(df_merged['Demand (MW)'], color='orange', ax=ax2)
ax2.set_ylabel('KDE', color='orange')

plt.show()

#### 3.8 Analyzing Correlation 
- Analyzing Correlation between Demand(MW) and other paramaters

In [ ]:
for column in df_merged.columns[3:18]:
    print(f"Correlation of Demand(MW) with {column}: {df_merged['Demand (MW)'].corr(df_merged[column])}")

- Plotting a heatmap of corrlation

In [ ]:
selected_columns = df_merged.columns[[1] + list(range(3, 10))]
correlation_matrix =df_merged[selected_columns].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='Greens', fmt=".2f", vmin=-1, vmax=1)

#### 3.9 Regrestion Plot of highly correlated datas
- Demand(MW) has high correlation with temperature and dewpoint whose regression plot are as follows.

In [ ]:
sns.regplot(x='Temperature', y='Demand (MW)', data=df_merged, scatter_kws={'s': 10}, line_kws={'color': 'green'})

In [ ]:
sns.regplot(x='dewpoint', y='Demand (MW)', data=df_merged, scatter_kws={'s': 10}, line_kws={'color': 'green'})

- Grouping data by month within each date and plotting a scatter plot between Temperature, dewpoint and Demand (MW)

In [ ]:
tempdf = df_merged
tempdf['datetime'] = pd.to_datetime(df_merged['datetime'])

numeric_columns = df_merged.select_dtypes(include=['number']).columns

# Grouping by date and then by month within each date, and calculating the mean for numeric columns
monthlydf = tempdf.groupby(tempdf['datetime'].dt.to_period("M"))[numeric_columns].mean().reset_index()

In [ ]:

sns.regplot(x='Temperature', y='Demand (MW)', data=monthlydf, scatter_kws={'s': 10}, line_kws={'color': 'green'})

In [ ]:
sns.regplot(x='dewpoint', y='Demand (MW)', data=monthlydf, scatter_kws={'s': 10}, line_kws={'color': 'green'})

#### 3.10 Dropping less significant data
Dropping less significant data after correlation analysis, i.e very low correlation as well as redundant data (i.e solarradiation and uv index where both have almost 1 correlation , here data with higher correlation is kept)


In [ ]:
print(df_merged["solarradiation"].corr(df_merged["uvindex"]))
print(df_merged["Temperature"].corr(df_merged["feelslike"]))

In [ ]:
df_merged.drop(['feelslike','uvindex', 'precipitation', 'sealevelpressure', 'snow', 'snowdepth', 'windspeed', 'winddirection'], inplace=True, axis=1)

#### 3.11 Visualization of categorical data
- Plotting conditions vs Demand(MW)

In [ ]:
plt.figure(figsize=(10, 5))
sns.barplot(x='conditions', y='Demand (MW)',hue='conditions', data=df_merged)

plt.title('Conditions vs Demand (MW)')
plt.xlabel('Conditions')
plt.ylabel('Demand (MW)')
plt.xticks(rotation=90)

plt.show()

### 4. Feature Engineering

#### [Zenml-MLfow pipeline for Feature Engineering](../steps/demand_forecasting/feature_engineering.py)

#### 4.1 Handling Categorical Data
Using pandas get dummies to handle categorical variables like 
condition creating new columns consisting of 0s and 1s for each columns 

In [ ]:
dummies = pd.get_dummies(df_merged['conditions'], prefix='overcast')
df_final = pd.concat([df_merged, dummies], axis=1)

In [ ]:
df_merged.iloc[:,3:10]

#### 4.2 Normalization
- Normalize continuous values and avoid vanishing gradient problems to finalize our data before model training.

In [ ]:

scaler = MinMaxScaler()
X = scaler.fit_transform(df_merged.iloc[:,3:10])

In [ ]:
df_merged.iloc[:,1]

In [ ]:
scaler = MinMaxScaler()
y = scaler.fit_transform(df_merged.iloc[:,1].values.reshape(-1,1))

#### 4.3 Train Validation Test Split
- Splitting the data into different splits

In [ ]:
def train_test_split(X, y, train_size=0.8):
    X_train = X[:int(train_size * len(X))]
    X_test = X[int(train_size * len(X)):]
    y_train = y[:int(train_size * len(y))]
    y_test = y[int(train_size * len(y)):]
    return X_train, X_test, y_train, y_test

- Separating dataset into train and test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)

### 5. Model Evaluation
#### [Zenml-MLfow pipeline for Model Evaluation](../steps/model_evaluaton/model_.py)
#### 5.1 Create and Evaluate Function
- Creating an evaulate function to give all metrics after model training

In [ ]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [ ]:
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGBRegressor": XGBRegressor(), 
    "CatBoosting Regressor": CatBoostRegressor(verbose=False),
    "AdaBoost Regressor": AdaBoostRegressor()
}
model_list = []
r2_list =[]

In [ ]:
def get_parameter_grid_for_model(model_name):
    if model_name == "Lasso":
        return {'alpha': [0.1, 1.0, 10.0]}

    elif model_name == "Ridge":
        return {'alpha': [0.1, 1.0, 10.0]}

    elif model_name == "K-Neighbors Regressor":
        return {'n_neighbors': [3, 5, 7]}

    elif model_name == "Decision Tree":
        return {'max_depth': [None, 5, 10], 'min_samples_split': [2, 5, 10]}

    elif model_name == "Random Forest Regressor":
        return {'n_estimators': [50, 100, 200], 'max_depth': [None, 5, 10], 'min_samples_split': [2, 5, 10]}

    elif model_name == "XGBRegressor":
        return {'n_estimators': [50, 100, 200], 'max_depth': [3, 5, 7], 'learning_rate': [0.01, 0.1, 0.2]}

    elif model_name == "CatBoosting Regressor":
        return {'iterations': [50, 100, 200], 'depth': [3, 5, 7], 'learning_rate': [0.01, 0.1, 0.2]}

    elif model_name == "AdaBoost Regressor":
        return {'n_estimators': [50, 100, 200], 'learning_rate': [0.01, 0.1, 0.2]}

    else:
        raise ValueError(f"Hyperparameter grid not defined for {model_name}")


#### 5.2 Hyperparameter Tuning
- This script evaluates and compares the performance of various  models

##### Models :
- Lasso
- Ridge
- K-Neighbors Regressor
- Decision Tree
- Random Forest Regressor
- XGBRegressor
- CatBoosting Regressor
- AdasBoost Regressor


In [ ]:


models = {
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGBRegressor": XGBRegressor(), 
    "CatBoosting Regressor": CatBoostRegressor(verbose=False),
    "AdaBoost Regressor": AdaBoostRegressor()
}

model_list = []
r2_list = []

for model_name, model in models.items():
    param_grid = get_parameter_grid_for_model(model_name)

    grid_search = GridSearchCV(model, param_grid, cv=5, scoring='r2')

    grid_search.fit(X_train, y_train)

    best_model = grid_search.best_estimator_

    y_train_pred = best_model.predict(X_train)
    y_test_pred = best_model.predict(X_test)

    model_train_mae, model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)
    model_test_mae, model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    print(model_name)
    print('Best hyperparameters:', grid_search.best_params_)
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))
    print('----------------------------------')
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)
    print('=' * 35)
    print('\n')

    model_list.append(model_name)


### 6. Model Training
#### [Zenml-MLfow pipeline for Data Preprocessing](../steps/demand_forecasting/model_training.py)

#### 6.1 XGBoost Regression Model Training and Saving

In [ ]:
xgb_model = XGBRegressor(n_estimators=1000, learning_rate=0.05, early_stopping_rounds=10)

In [ ]:
xgb_model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False)

In [ ]:
joblib.dump(xgb_model, '../model/demand_forecasting.pkl')

In [ ]:
xgb_pred = xgb_model.predict(X_test)

### 7. Model Evaluation
 After training the XGBoost regression model for demand forecasting, following metrics are used to assess the performance : 

 - Mean Squared Error 
 - Mean Absolute Error
 - R2 Score

In [ ]:
print("mean squared error:", mean_squared_error(y_test, xgb_pred))
print("mean absolute error:",mean_absolute_error(y_test, xgb_pred))
print("r2 score:",r2_score(y_test, xgb_pred))